<a href="https://colab.research.google.com/github/dagnybrand/AI-news-bias/blob/main/embedding_text_data_AI_News_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

with open('bias_data_short.json') as f:
  og_data_dict = json.loads(f.read())


In [ ]:
print(og_data_dict[0])

# will use these sections from JSON file
headers = ['topic', 'source', 'bias', 'title', 'content', ]

{'topic': 'politics', 'source': 'Politico', 'bias': 0, 'url': 'https://www.politico.com/news/magazine/2019/11/13/trump-impeachment-hearing-change-mind-congress-070145', 'title': 'Can the Impeachment Hearings Actually Change Anyone’s Mind?', 'date': '2019-11-13', 'authors': '', 'content': 'This happens for different reasons , but a key element is the vicious cycle between holding strong attitudes on an issue and something called “ selective perception. ” Essentially , the stronger your views are on an issue like Trump ’ s impeachment , the more likely you are to attend more carefully to information that supports your views and to ignore or disregard information that contradicts them . Consuming more belief-consistent information will , in turn , increase your original support or disapproval for impeachment , which just fortifies your attitudes . So , no , not much change will be seen in the minds of the 33 percent .\nExcept , maybe . One of the more interesting findings from research on

In [ ]:
# create data base with given headers
import pandas as pd

df = pd.DataFrame(index=headers)

print(df)

Empty DataFrame
Columns: []
Index: [topic, source, bias, title, content]


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=2f33081611746bc23549eb11b58f49ee8207cf27f59a721d290854f82e91a5ae
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
!pip3 install tensorflow-hub

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
topic_embeddings = model([entry['topic'] for entry in tqdm(og_data_dict)])

100%|██████████| 1001/1001 [00:00<00:00, 436162.30it/s]


In [ ]:
print(topic_embeddings)

tf.Tensor(
[[-0.06286574 -0.04393262 -0.01329987 ... -0.06114992 -0.00552988
  -0.01565152]
 [-0.03738969 -0.0498386   0.05589052 ... -0.04552171 -0.0534011
  -0.06769715]
 [ 0.01772067 -0.01596603  0.01255758 ... -0.04339546 -0.02090789
  -0.06721355]
 ...
 [ 0.00364689  0.03364109  0.08796966 ... -0.03458311  0.05572385
  -0.08182494]
 [-0.02808105 -0.00582345  0.07537998 ... -0.05297739  0.02434535
  -0.06205724]
 [ 0.06156581 -0.0334102   0.06232018 ...  0.0048111   0.05785914
  -0.03734188]], shape=(1001, 512), dtype=float32)


In [ ]:
source_embeddings = model([entry['source'] for entry in tqdm(og_data_dict)])

100%|██████████| 1001/1001 [00:00<00:00, 490822.81it/s]


In [ ]:
title_embeddings = model([entry['title'] for entry in tqdm(og_data_dict)])

100%|██████████| 1001/1001 [00:00<00:00, 248108.87it/s]


In [ ]:
content_embeddings = model([entry['content'] for entry in tqdm(og_data_dict)])

100%|██████████| 1001/1001 [00:00<00:00, 322242.56it/s]


In [ ]:
print(topic_embeddings.shape, source_embeddings.shape, title_embeddings.shape, content_embeddings.shape)

(1001, 512) (1001, 512) (1001, 512) (1001, 512)


In [ ]:
# each entry has 512 vector points
# can I train using vectors?
embedded_data = {}
embedded_data['titles'] = title_embeddings
embedded_data['topics'] = topic_embeddings
embedded_data['sources'] = source_embeddings
embedded_data['content'] = content_embeddings
embedded_data['biases'] = [int(entry['bias']) for entry in og_data_dict]

In [ ]:
for key in embedded_data:
  np_array = np.array(embedded_data[key])
  list_array = np_array.tolist()
  embedded_data[key] = list_array


In [ ]:
with open('embedded.json', 'w') as f:
  json.dump(embedded_data, f)

In [ ]:
# use Sentence Embedding to get numerical values for entries

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
# testing model
sentence_embedding = model.encode("hi")
print(len(sentence_embedding))

768


In [ ]:
# really slow compared to universal embedder

from tqdm import tqdm

# for each article, get embedding of topic, title, source, and content
# store data in data base

em_topics = []
em_sources = []
em_titles = []
em_contents = []
biases = []

for entry in tqdm(og_data_dict):
  em_topic = model.encode(entry['topic'])
  em_source = model.encode(entry['source'])
  em_title = model.encode(entry['title'])
  em_content = model.encode(entry['content'])

  em_topics.append(em_topic)
  em_sources.append(em_source)
  em_titles.append(em_title)
  em_contents.append(em_contents)
  biases.append(int(entry['bias']))


#df.assign(topic=em_topics, source=em_sources, title=em_titles, content=em_content, bias=biases)

 71%|███████   | 710/1001 [09:51<04:02,  1.20it/s]


KeyboardInterrupt: ignored

In [ ]:
# since the sentence embedding takes a while, store the values in a file
embedded_dict = {}

embedded_dict['topics'] = em_topics
embedded_dict['sources'] = em_sources
embedded_dict['titles'] = em_titles
embedded_dict['content'] = em_contents
embedded_dict['bias'] = biases

In [ ]:
embedded_df = pd.DataFrame(embedded_dict)

embedded_json = embedded_df.to_json(orient='index')

NameError: ignored

In [ ]:
import numpy
from json import JSONEncoder

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

with open('embedded.json', 'w') as f:
  json.dump(embedded_dict, f, cls=NumpyArrayEncoder)

ValueError: ignored

In [ ]:
import sklearn
from sklearn.preprocessing import StandardScaler